In [29]:
#Test input ANN
import keras
from keras.models import load_model
import keras.backend as K
from snntoolbox.parsing.utils import get_custom_activations_dict

def relu6(x):
    return K.relu(x, max_value=6)

from keras.datasets import cifar10
from keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_test = to_categorical(y_test, 10)
y_train = to_categorical(y_train, 10)
x_test = x_test/255
x_train = x_train/255


In [30]:
parsed_model_path = '/mnt/snntoolbox/snn_toolbox_private/examples/models/05-mobilenet_dwarf_v1/mobilenet_dwarf_v1_parsed.h5'
unparsed_model_path = '/mnt/snntoolbox/snn_toolbox_private/examples/models/05-mobilenet_dwarf_v1/mobilenet_dwarf_v1.h5'
custom_objects_path = '/mnt/snntoolbox/snn_toolbox_private/examples/models/05-mobilenet_dwarf_v1/custom_objects.json'
parsed_model = load_model(parsed_model_path, get_custom_activations_dict(custom_objects_path))
unparsed_model = load_model(unparsed_model_path, custom_objects={'relu6':relu6})

In [31]:
unparsed_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 32, 32, 32)        320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)       

In [23]:
unparsed_score = unparsed_model.evaluate(x_test, y_test, batch_size = 1)
parsed_score = parsed_model.evaluate(x_test, y_test, batch_size = 1)

10000/10000 [==============================] - 32s 3ms/step


In [24]:
print(dict(zip(unparsed_model.metrics_names, unparsed_score)))
print(dict(zip(parsed_model.metrics_names, parsed_score)))


{'top_k_categorical_accuracy': 1.0, 'loss': nan, 'acc': 0.6758}
{'loss': 0.9287359311230518, 'acc': 0.6962}


In [ ]:
#Maxpooling to softmax experiment

In [25]:
fine_tuned_model = unparsed_model

In [26]:
fine_tuned_model.layers[:-6]

In [27]:
#for layer in fine_tuned_model.layers[:-6]:
    #layer.trainable = True

In [28]:
fine_tuned_model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
fine_tuned_model.fit(x_train, y_train, epochs = 5, batch_size= 1)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/5
50000/50000 [==============================] - 983s 20ms/step - loss: 1.3984 - acc: 0.4924
Epoch 2/5
 9968/50000 [====>.........................] - ETA: 13:04 - loss: 1.1143 - acc: 0.6047

KeyboardInterrupt: 

In [ ]:
fine_tuned_score = fine_tuned_model.evaluate(x_test, y_test, batch_size = 1)
print(dict(zip(fine_tuned_model.metrics_names, fine_tuned_score)))


In [ ]:
fine_tuned_model.save('fine_tuned_model.h5')